In [ ]:
'''
In this code, we remove the clutter and background of Lung CT images.  
'''

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
        print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
import cv2
import glob

In [ ]:
#Standardize the pixel values
def background_removed(img, display=False,erosion = 2,dilation = 1):
    cropping_error = False
    # Find the average pixel value close to foreground to renormalize washed out images 
    img= cv2.copyMakeBorder(img,1,1,0,0,cv2.BORDER_WRAP)
    img= cv2.copyMakeBorder(img,25,25,25,25,cv2.BORDER_CONSTANT,value=0)
    
    row_size= img.shape[0]
    col_size = img.shape[1]
    
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    
    # finding the threshold to separate foreground (soft tissue / bone) and background (lung/air)
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1])) # kmeans
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate 
    # to include some of the pixels surrounding the lung.  
    # We want to aviod accidentally cliping the lung.
    eroded = morphology.erosion(thresh_img,np.ones([erosion,erosion]))
    dilated = morphology.dilation(eroded,np.ones([dilation,dilation]))
      
    h, w = dilated.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    dilated =dilated.astype('uint8')
    foreground1 = 1- dilated  # first foreground mask
    
    # Find the largest contour in intermediate image
    cnts, _ = cv2.findContours(foreground1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = max(cnts, key=cv2.contourArea)
    
    foreground2 = np.zeros(img.shape, np.uint8)
    cv2.drawContours(foreground2, [cnt], -1, 255, cv2.FILLED) # second foreground mask
    
    # Intersection of the two foreground masks 
    foreground = cv2.bitwise_and(foreground1, foreground2)  # Bitwise AND operation of the two images 
    
    # filling the holes in the enclosed mask   
    cv2.floodFill(foreground,mask, (0,0), 1) 

    masked = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')  
    masked = (1-mask[:row_size,:col_size]) * masked # Applying the mask on the image
    
    # Cropping the masked image 
    _,thresh = cv2.threshold(masked,1,255,cv2.THRESH_BINARY)
    contours , hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x),reverse = True)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    crop = masked[y:y+h,x:x+w]
    mas = 1-mask[:row_size,:col_size]
    crop_mask = mas[y:y+h,x:x+w]
    
    nonzero = np.where ((crop_mask ==1))
    crop[nonzero] = ((crop[nonzero] - crop[nonzero].min()) * (1/(crop[nonzero].max() - crop[nonzero].min()) * 255)).astype('uint8')
    
    if crop.shape[0] * crop.shape[1] < int( row_size * col_size / 27) :
        cropping_error = True
           
    return crop , cropping_error

In [ ]:
image_list = glob.glob('/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data/2COVID'+'/*.png')
count = 0
for i in range(200,len(image_list)):
            image = image_list[i]  
            count += 1
            image2 = '/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data/2COVID/'+image.split('/')[-1]  
            original = cv2.imread(image)
            original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)  
            try: 
                    plt.figure(figsize=(10,20))  # change the size of figure!
                    plt.subplot(1, 2, 1)
                    img , cropping_error = background_removed(original)
                    if cropping_error:
                        print('The code did not work properly on this image named: ',image.split('/')[-1], 'and original image is outputted:')
                        img = original
                    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
                    plt.title('Original',fontsize= 10)
                    plt.subplot(1, 2, 2)
                    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title('Background removed',fontsize= 10)
                    #plt.suptitle(image.split('/')[-1],fontsize= 15)
                    plt.show()
            except:
                    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
                    plt.title('The code did not work on this image file named: ',image.split('/')[-1],fontsize= 10)
                    plt.show()
                    
                   
            if count == 200 :
                break
            